In [1]:
import os
import pickle
import numpy as np

from algos import *
from infect import infect
from sbm import SBM

In [2]:
def create_communities(s, N, M):
    size = round(N/M)
    communities = []

    for i in range(M):
        if(i == M-1):
            comm = s[i*size:]
        else:
            comm = s[i*size:(i+1)*(size)]
        communities.append(comm)

    return communities

In [3]:
def alligator(communities):
    # can delete i at end: is only used for printing indices
    i = 0
    for cult in communities:
        print(cult)
        # each cult is a subset of infected (or not) individuals
        Qtesting1(cult)
        i+=1

In [5]:
N = 30
M = 5
q0 = .4
q1 = 0

p0 = .1
p1 = .5
time_steps = 2

G = SBM(N, M, q0,q1)

s = infect(G, p0, p1, time_steps)
comms = create_communities(s, N, M)
for c in range(len(comms)):
    print("Community ", c, " -- ", comms[c])

alligator(comms)

Original infected individuals:  [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Infected individuals after step  0  -  [0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Infected individuals after step  1  -  [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Community  0  --  [0. 1. 0. 1. 0. 1.]
Community  1  --  [0. 1. 0. 1. 1. 1.]
Community  2  --  [0. 1. 0. 0. 1. 1.]
Community  3  --  [0. 0. 0. 0. 0. 0.]
Community  4  --  [0. 0. 0. 0. 0. 0.]
[0. 1. 0. 1. 0. 1.]
RUNNING TEST ON original:  [0. 1. 0. 1. 0. 1.] 


TESTING group:  [0. 1. 0. 1. 0. 1.]

TESTING group:  [0. 1.]
--Num binary tests:  2
--Num binary stages:  2
Setting stages to  4

TESTING group:  [0. 1.]
--Num binary tests:  2
--Num binary stages:  2
Setting stages to  4

TESTING group:  [0. 1.]
--Num binary tests:  2
--Num binary stages:  2
Setting stages to  4
[0. 1. 0. 1. 1. 1.]
RUNNING TEST ON 